# Baseline Specification for Causal Identification: Static vs GP Time-Varying Intercepts

This notebook investigates how **baseline specification** affects both predictive performance and causal parameter recovery in Media Mix Models (MMM).

## The Fundamental Question

In MMM, we decompose sales into:
```
Sales = Baseline + Marketing Effects + Controls + Noise
```

**The identification problem**: If the baseline is too rigid, seasonal patterns get misattributed to marketing. If too flexible, the baseline absorbs true marketing effects.

## Why This Matters for Causality

The baseline is a **nuisance function** — we don't make business decisions based on baseline values. However:
- **Better baseline modeling → Better marketing effect isolation**
- **Over-flexible baseline → Marketing effects absorbed → Poor causal identification**

This is fundamentally different from optimizing priors on marketing effects (which would be philosophically problematic).

## Four Baseline Strategies

We compare:

1. **Static intercept only** - Constant baseline, no seasonality
2. **Static + Fourier seasonality** - Traditional MMM approach
3. **GP flexible baseline** - Gaussian Process captures all temporal variation
4. **GP + Fourier hybrid** - Fourier for regular patterns, GP for irregular events

## Evaluation Framework

We evaluate on **dual objectives**:
- **Predictive**: Test set CRPS (out-of-sample accuracy)
- **Causal**: ROAS error vs ground truth (parameter recovery)

**Key insight**: The best predictive model may not be the best causal model!

## Setup

In [1]:
import json
import time
import warnings
from pathlib import Path
from typing import Any

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
from pymc_marketing.metrics import crps
from pymc_marketing.mmm import GeometricAdstock, LogisticSaturation, MMM
from rich import print as rprint
from rich.console import Console
from rich.table import Table

# Set random seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Plotting style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (14, 6)

rprint("[bold green]Setup complete![/bold green]")

Setup complete!

## Configuration

In [27]:
# Train/test split (same as notebook 03)
TEST_SIZE_WEEKS = 24

# Fixed hyperparameters from notebook 03 CRPS optimization
FIXED_HYPERPARAMS = {
    "adstock_max_lag": 10,
}

# MCMC settings for comparison phase (faster)
COMPARISON_MCMC = {
    "draws": 1000,
    "tune": 1000,
    "chains": 4,
    "nuts_sampler": "numpyro",
    "random_seed": RANDOM_SEED,
    "progressbar": False,
}

# MCMC settings for final model (production quality)
FINAL_MCMC = {
    "draws": 2000,
    "tune": 2000,
    "chains": 4,
    "nuts_sampler": "numpyro",
    "random_seed": RANDOM_SEED,
    "progressbar": False,
}

# Convergence thresholds
COMPARISON_CONVERGENCE = {
    "divergence_threshold": 0.10,
    "rhat_threshold": 1.10,
    "ess_threshold": 50,
}

FINAL_CONVERGENCE = {
    "divergence_threshold": 0.01,
    "rhat_threshold": 1.01,
    "ess_threshold": 400,
}

# Channel and control columns
CHANNEL_COLUMNS = [
    "x1_Search-Ads",
    "x2_Social-Media",
    "x3_Local-Ads",
    "x4_Email",
]
CONTROL_COLUMNS = ["c1", "c2"]

rprint("[bold blue]Configuration:[/bold blue]")
rprint(f"Train/test split: {TEST_SIZE_WEEKS} weeks for test")
rprint(f"Adstock max lag: {FIXED_HYPERPARAMS['adstock_max_lag']} (from notebook 03)")
rprint(f"Comparison MCMC: {COMPARISON_MCMC['draws']} draws, {COMPARISON_MCMC['chains']} chains")
rprint(f"Final MCMC: {FINAL_MCMC['draws']} draws, {FINAL_MCMC['chains']} chains")

Configuration:

Train/test split: 24 weeks for test

Adstock max lag: 10 (from notebook 03)

Comparison MCMC: 1000 draws, 4 chains

Final MCMC: 2000 draws, 4 chains

## Baseline Strategy Definitions

In [28]:
BASELINE_STRATEGIES = [
    {
        "name": "static_intercept_only",
        "description": "Constant baseline, no seasonality",
        "config": {
            "yearly_seasonality": None,
            "time_varying_intercept": False,
        },
        "hypothesis": "Simplest model - may attribute seasonal/trend patterns to marketing",
    },
    {
        "name": "static_fourier_seasonality",
        "description": "Static baseline with Fourier seasonality",
        "config": {
            "yearly_seasonality": 3,  # From notebook 03 CRPS optimization
            "time_varying_intercept": False,
        },
        "hypothesis": "Standard MMM approach - handles regular seasonal patterns",
    },
    {
        "name": "gp_flexible_baseline",
        "description": "GP baseline without Fourier (GP handles all temporal variation)",
        "config": {
            "yearly_seasonality": None,
            "time_varying_intercept": True,
        },
        "hypothesis": "Maximally flexible - risk of absorbing marketing signal",
    },
    {
        "name": "gp_plus_fourier",
        "description": "GP baseline WITH Fourier seasonality",
        "config": {
            "yearly_seasonality": 3,
            "time_varying_intercept": True,
        },
        "hypothesis": "Best of both worlds - Fourier for regular patterns, GP for irregular",
    },
]

# Display strategies
console = Console()
table = Table(title="Baseline Strategies to Compare", show_header=True, header_style="bold magenta")
table.add_column("Strategy", style="cyan")
table.add_column("Description")
table.add_column("Fourier", justify="center")
table.add_column("GP", justify="center")
table.add_column("Hypothesis")

for strategy in BASELINE_STRATEGIES:
    fourier = str(strategy["config"]["yearly_seasonality"]) if strategy["config"]["yearly_seasonality"] else "✗"
    gp = "✓" if strategy["config"]["time_varying_intercept"] else "✗"
    table.add_row(
        strategy["name"],
        strategy["description"],
        fourier,
        gp,
        strategy["hypothesis"],
    )

console.print(table)

                                          Baseline Strategies to Compare                                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Strategy                   ┃ Description                      ┃ Fourier ┃ GP ┃ Hypothesis                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ static_intercept_only      │ Constant baseline, no            │    ✗    │ ✗  │ Simplest model - may attribute   │
│                            │ seasonality                      │         │    │ seasonal/trend patterns to       │
│                            │                                  │         │    │ marketing                        │
│ static_fourier_seasonality │ Static baseline with Fourier     │    3    │ ✗  │ Standard MMM approach - handles  │
│                            │ seasonality                      │         │    │ regular seasonal patterns        │
│ gp_flexible_baseline       │ GP baseline without Fourier (GP  │    ✗    │ ✓  │ Maximally flexible - risk of     │
│                            │ handles all temporal variation)  │         │    │ absorbing marketing signal       │
│ gp_plus_fourier            │ GP baseline WITH Fourier         │    3    │ ✓  │ Best of both worlds - Fourier    │
│                            │ seasonality                      │         │    │ for regular patterns, GP for     │
│                            │                                  │         │    │ irregular                        │
└────────────────────────────┴──────────────────────────────────┴─────────┴────┴──────────────────────────────────┘

## Load and Prepare Data

In [29]:
def load_mmm_data(data_path: str | Path) -> pl.DataFrame:
    """Load MMM data from CSV file.
    
    Args:
        data_path: Path to the mmm_data.csv file
        
    Returns:
        Polars DataFrame with parsed date column
    """
    return pl.read_csv(data_path).with_columns(pl.col("date").str.to_date())


def split_train_test(
    df: pl.DataFrame, test_size_weeks: int
) -> tuple[pl.DataFrame, pl.DataFrame]:
    """Split data into train and test sets chronologically.
    
    Args:
        df: Full dataset
        test_size_weeks: Number of weeks to use for test set
        
    Returns:
        Tuple of (train_df, test_df)
    """
    n_total = df.shape[0]
    n_train = n_total - test_size_weeks
    df_sorted = df.sort("date")
    return df_sorted[:n_train], df_sorted[n_train:]


# Load data
data_path = Path("../data/mmm_data.csv")
df = load_mmm_data(data_path)

rprint(f"[bold green]Data loaded successfully[/bold green]")
rprint(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
rprint(f"Date range: {df['date'].min()} to {df['date'].max()}")

# Split data
df_train, df_test = split_train_test(df, TEST_SIZE_WEEKS)

rprint(f"\n[bold blue]Train/Test Split:[/bold blue]")
rprint(
    f"Train set: {df_train.shape[0]} weeks ({df_train['date'].min()} to {df_train['date'].max()})"
)
rprint(
    f"Test set:  {df_test.shape[0]} weeks ({df_test['date'].min()} to {df_test['date'].max()})"
)

# Convert to pandas (PyMC-Marketing requires pandas)
df_pandas = df.to_pandas()
df_train_pandas = df_train.to_pandas()
df_test_pandas = df_test.to_pandas()

# Prepare train/test splits
X_train = df_train_pandas.drop(columns=["y"])
y_train = df_train_pandas["y"]
X_test = df_test_pandas.drop(columns=["y"])
y_test = df_test_pandas["y"]
X_full = df_pandas.drop(columns=["y"])
y_full = df_pandas["y"]

# Load ground truth
ground_truth_path = Path("../data/ground_truth_parameters.json")
with open(ground_truth_path) as f:
    ground_truth = json.load(f)

rprint("[bold green]Data preparation complete![/bold green]")

Data loaded successfully

Shape: 104 rows × 11 columns

Date range: 2020-01-05 to 2021-12-26

Train/Test Split:

Train set: 80 weeks (2020-01-05 to 2021-07-11)

Test set:  24 weeks (2021-07-18 to 2021-12-26)

Data preparation complete!

## Core Functions: Model Creation and Evaluation

In [30]:
def create_mmm_with_baseline(
    baseline_config: dict[str, Any],
    channel_columns: list[str],
    control_columns: list[str],
    adstock_max_lag: int,
) -> MMM:
    """Create MMM with specified baseline configuration.
    
    Args:
        baseline_config: Dict with 'yearly_seasonality' and 'time_varying_intercept'
        channel_columns: Marketing channel names
        control_columns: Control variable names
        adstock_max_lag: Maximum lag for adstock transformation
        
    Returns:
        Configured (unfitted) MMM instance
    """
    return MMM(
        date_column="date",
        channel_columns=channel_columns,
        control_columns=control_columns,
        adstock=GeometricAdstock(l_max=adstock_max_lag),
        saturation=LogisticSaturation(),
        yearly_seasonality=baseline_config["yearly_seasonality"],
        time_varying_intercept=baseline_config["time_varying_intercept"],
    )


def check_convergence(
    mmm: MMM,
    divergence_threshold: float,
    rhat_threshold: float,
    ess_threshold: float,
    strategy_name: str | None = None,
) -> tuple[bool, dict[str, float]]:
    """Check MCMC convergence diagnostics.
    
    Args:
        mmm: Fitted MMM model
        divergence_threshold: Maximum allowed divergence rate (0-1)
        rhat_threshold: Maximum allowed R-hat value
        ess_threshold: Minimum required effective sample size
        strategy_name: Optional strategy name for logging
        
    Returns:
        Tuple of (converged: bool, diagnostics: dict)
    """
    # Check divergences
    n_divergences = int(mmm.idata.sample_stats.diverging.sum().item())
    n_draws = mmm.idata.posterior.sizes["draw"]
    n_chains = mmm.idata.posterior.sizes["chain"]
    total_samples = n_draws * n_chains
    divergence_rate = n_divergences / total_samples

    # Check R-hat
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore", category=RuntimeWarning, message=".*invalid value encountered.*"
        )
        rhat = az.rhat(mmm.idata)
        max_rhat = float(rhat.to_array().max())

    # Check ESS
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore", category=RuntimeWarning, message=".*invalid value encountered.*"
        )
        ess = az.ess(mmm.idata)
        min_ess = float(ess.to_array().min())

    # Convergence checks
    divergence_ok = divergence_rate <= divergence_threshold
    rhat_ok = max_rhat <= rhat_threshold
    ess_ok = min_ess >= ess_threshold
    converged = divergence_ok and rhat_ok and ess_ok

    diagnostics = {
        "n_divergences": n_divergences,
        "divergence_rate": divergence_rate,
        "max_rhat": max_rhat,
        "min_ess": min_ess,
    }

    # Logging
    name_str = f"[{strategy_name}]" if strategy_name else "Model"
    if converged:
        rprint(
            f"[green]✓ {name_str} CONVERGED:[/green] "
            f"div={divergence_rate*100:.1f}%, rhat={max_rhat:.3f}, ess={min_ess:.0f}"
        )
    else:
        rprint(
            f"[red]✗ {name_str} FAILED:[/red] "
            f"div={divergence_rate*100:.1f}%, rhat={max_rhat:.3f}, ess={min_ess:.0f}"
        )

    return converged, diagnostics


def compute_test_crps(mmm: MMM, X_test: pd.DataFrame, y_test: pd.Series) -> float:
    """Compute CRPS on test set using posterior predictive.
    
    Args:
        mmm: Fitted MMM model
        X_test: Test features
        y_test: Test target values
        
    Returns:
        CRPS score (lower is better)
    """
    # Sample posterior predictive
    mmm.sample_posterior_predictive(X_test, original_scale=True, extend_idata=True)

    # Extract and rescale
    y_pred_samples = mmm.idata.posterior_predictive["y"].values
    target_scale = float(mmm.idata.constant_data["target_scale"].values)
    y_pred_rescaled = y_pred_samples * target_scale

    # Reshape for CRPS
    n_chains, n_draws, n_obs = y_pred_rescaled.shape
    y_pred_reshaped = y_pred_rescaled.reshape(n_chains * n_draws, n_obs)

    return float(crps(y_test.values, y_pred_reshaped))


rprint("[bold green]Core functions defined![/bold green]")

Core functions defined!

## ROAS and Attribution Metrics

In [31]:
def compute_roas_metrics(
    mmm: MMM,
    channel_spend: pd.DataFrame,
    channel_columns: list[str],
    ground_truth: dict,
) -> dict[str, float]:
    """Compute ROAS and compare with ground truth.
    
    Args:
        mmm: Fitted MMM model
        channel_spend: DataFrame with channel spend columns
        channel_columns: Channel names
        ground_truth: Ground truth parameters
        
    Returns:
        Dictionary with ROAS metrics
    """
    # Compute contributions in original scale
    contributions = mmm.compute_mean_contributions_over_time(original_scale=True)

    # Compute ROAS
    total_contributions = contributions[channel_columns].sum()
    total_spend = channel_spend[channel_columns].sum()
    estimated_roas = total_contributions / total_spend

    # Get true ROAS
    true_roas_dict = ground_truth["roas_values"]["Local"]

    # Compute errors
    errors = {}
    errors_pct = {}
    absolute_errors = []
    percentage_errors = []

    for channel in channel_columns:
        channel_name = channel.split("_", 1)[1] if "_" in channel else channel
        est = estimated_roas[channel]
        true = true_roas_dict.get(channel_name, 0.0)

        error = abs(est - true)
        error_pct = (abs(est - true) / true * 100) if true != 0 else 0.0

        errors[f"roas_{channel_name.lower().replace('-', '_')}"] = est
        errors_pct[f"roas_{channel_name.lower().replace('-', '_')}_error_pct"] = error_pct

        absolute_errors.append(error)
        percentage_errors.append(error_pct)

    metrics = {
        "roas_mae": float(np.mean(absolute_errors)),
        "roas_mape": float(np.mean(percentage_errors)),
        **errors,
        **errors_pct,
    }

    return metrics


def compute_attribution_metrics(
    mmm: MMM, X: pd.DataFrame, y: pd.Series, channel_columns: list[str]
) -> dict[str, float]:
    """Compute marketing attribution share.
    
    Args:
        mmm: Fitted MMM model
        X: Features
        y: Target
        channel_columns: Channel names
        
    Returns:
        Dictionary with attribution metrics
    """
    # Get contributions
    contributions = mmm.compute_mean_contributions_over_time(original_scale=True)

    # Marketing contribution
    marketing_contrib = contributions[channel_columns].sum(axis=1).mean()
    total_sales = y.mean()
    marketing_share = marketing_contrib / total_sales

    return {
        "marketing_share": float(marketing_share),
        "baseline_share": float(1 - marketing_share),
        "total_sales_mean": float(total_sales),
        "marketing_sales_mean": float(marketing_contrib),
    }


rprint("[bold green]ROAS and attribution functions defined![/bold green]")

ROAS and attribution functions defined!

## Main Evaluation Pipeline

In [32]:
def fit_and_evaluate_strategy(
    strategy: dict[str, Any],
    X_train: pd.DataFrame,
    y_train: pd.Series,
    X_test: pd.DataFrame,
    y_test: pd.Series,
    X_full: pd.DataFrame,
    y_full: pd.Series,
    ground_truth: dict,
    channel_columns: list[str],
    control_columns: list[str],
    adstock_max_lag: int,
    convergence_thresholds: dict,
    **fit_kwargs,
) -> tuple[dict[str, float], MMM | None]:
    """Complete evaluation pipeline for one baseline strategy.
    
    Args:
        strategy: Strategy dict with name, config, hypothesis
        X_train, y_train: Training data
        X_test, y_test: Test data
        X_full, y_full: Full dataset (for attribution computation)
        ground_truth: Ground truth parameters
        channel_columns: Channel names
        control_columns: Control variable names
        adstock_max_lag: Maximum adstock lag
        convergence_thresholds: Dict with convergence criteria
        **fit_kwargs: MCMC settings
        
    Returns:
        Tuple of (metrics dict, fitted MMM or None if failed)
    """
    rprint(f"\n[bold cyan]Evaluating: {strategy['name']}[/bold cyan]")
    rprint(f"Description: {strategy['description']}")
    rprint(f"Hypothesis: {strategy['hypothesis']}")

    try:
        # Create model
        start_time = time.time()
        mmm = create_mmm_with_baseline(
            baseline_config=strategy["config"],
            channel_columns=channel_columns,
            control_columns=control_columns,
            adstock_max_lag=adstock_max_lag,
        )

        # Fit on training set
        rprint("Fitting model on training set...")
        mmm.fit(X=X_train, y=y_train, **fit_kwargs)
        fit_time = time.time() - start_time

        # Check convergence
        converged, convergence_diagnostics = check_convergence(
            mmm=mmm,
            strategy_name=strategy["name"],
            **convergence_thresholds,
        )

        if not converged:
            rprint(f"[yellow]Warning: Model did not converge, but continuing...[/yellow]")

        # Compute test CRPS
        rprint("Computing test CRPS...")
        test_crps = compute_test_crps(mmm, X_test, y_test)
        rprint(f"Test CRPS: {test_crps:.2f}")

        # Compute train CRPS for overfitting check
        rprint("Computing train CRPS...")
        train_crps = compute_test_crps(mmm, X_train, y_train)
        rprint(f"Train CRPS: {train_crps:.2f}")

        # Compute ROAS metrics
        rprint("Computing ROAS metrics...")
        roas_metrics = compute_roas_metrics(
            mmm, X_full, channel_columns, ground_truth
        )
        rprint(f"ROAS MAE: {roas_metrics['roas_mae']:.2f}")

        # Compute attribution metrics
        rprint("Computing attribution metrics...")
        attribution_metrics = compute_attribution_metrics(
            mmm, X_full, y_full, channel_columns
        )
        rprint(f"Marketing share: {attribution_metrics['marketing_share']*100:.1f}%")

        # Combine all metrics
        metrics = {
            "strategy": strategy["name"],
            "hypothesis": strategy["hypothesis"],
            "train_crps": train_crps,
            "test_crps": test_crps,
            "crps_overfit_ratio": test_crps / train_crps if train_crps > 0 else 1.0,
            **roas_metrics,
            **attribution_metrics,
            "converged": converged,
            **convergence_diagnostics,
            "fit_time_seconds": fit_time,
        }

        rprint(f"[green]✓ {strategy['name']} evaluation complete![/green]")
        return metrics, mmm

    except Exception as e:
        rprint(f"[red]✗ {strategy['name']} failed: {e}[/red]")
        return {
            "strategy": strategy["name"],
            "hypothesis": strategy["hypothesis"],
            "status": "failed",
            "error": str(e),
        }, None


rprint("[bold green]Evaluation pipeline defined![/bold green]")

Evaluation pipeline defined!

## Run Systematic Comparison

This will fit all 4 baseline strategies and evaluate each on train/test sets.

**Note**: This takes approximately 15-20 minutes to complete.

In [33]:
# Run comparison
results = []
models = {}

rprint("\n[bold magenta]Starting systematic baseline comparison...[/bold magenta]")
rprint(f"Evaluating {len(BASELINE_STRATEGIES)} strategies\n")

for strategy in BASELINE_STRATEGIES:
    metrics, mmm = fit_and_evaluate_strategy(
        strategy=strategy,
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        y_test=y_test,
        X_full=X_full,
        y_full=y_full,
        ground_truth=ground_truth,
        channel_columns=CHANNEL_COLUMNS,
        control_columns=CONTROL_COLUMNS,
        adstock_max_lag=FIXED_HYPERPARAMS["adstock_max_lag"],
        convergence_thresholds=COMPARISON_CONVERGENCE,
        **COMPARISON_MCMC,
    )
    results.append(metrics)
    if mmm is not None:
        models[strategy["name"]] = mmm

# Convert to DataFrame
results_df = pl.DataFrame(results)

rprint("\n[bold green]Comparison complete![/bold green]")
rprint(f"Successfully evaluated {len(models)}/{len(BASELINE_STRATEGIES)} strategies")

Starting systematic baseline comparison...

Evaluating 4 strategies

Evaluating: static_intercept_only

Description: Constant baseline, no seasonality

Hypothesis: Simplest model - may attribute seasonal/trend patterns to marketing

...

Computing attribution metrics...

Marketing share: 62.3%

✓ gp_plus_fourier evaluation complete!

Comparison complete!

Successfully evaluated 4/4 strategies

## Results Table

In [34]:
# Create comprehensive results table
console = Console()
table = Table(
    title="Baseline Strategy Comparison Results",
    show_header=True,
    header_style="bold cyan",
)

table.add_column("Strategy", style="cyan")
table.add_column("Test CRPS", justify="right")
table.add_column("ROAS MAE", justify="right")
table.add_column("ROAS MAPE", justify="right")
table.add_column("Marketing %", justify="right")
table.add_column("Converged", justify="center")
table.add_column("Time (s)", justify="right")

for row in results_df.iter_rows(named=True):
    if "error" not in row:
        converged_symbol = "✓" if row["converged"] else "✗"
        converged_color = "green" if row["converged"] else "red"

        table.add_row(
            row["strategy"],
            f"{row['test_crps']:.2f}",
            f"{row['roas_mae']:.2f}",
            f"{row['roas_mape']:.1f}%",
            f"{row['marketing_share']*100:.1f}%",
            f"[{converged_color}]{converged_symbol}[/{converged_color}]",
            f"{row['fit_time_seconds']:.0f}",
        )
    else:
        table.add_row(
            row["strategy"],
            "[red]FAILED[/red]",
            "-",
            "-",
            "-",
            "[red]✗[/red]",
            "-",
        )

console.print("\n")
console.print(table)

# Ground truth reference
true_marketing_share = (
    sum(ground_truth["attribution_percentages"]["Local"].values()) / 100
)
rprint(f"\n[bold]Ground Truth Reference:[/bold]")
rprint(f"True marketing share: {true_marketing_share*100:.1f}%")

                                 Baseline Strategy Comparison Results                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Strategy                   ┃ Test CRPS ┃ ROAS MAE ┃ ROAS MAPE ┃ Marketing % ┃ Converged ┃ Time (s) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━┩
│ static_intercept_only      │    799.80 │     2.62 │     21.8% │       34.3% │     ✓     │        7 │
│ static_fourier_seasonality │    301.35 │     1.62 │     24.5% │       34.5% │     ✓     │       10 │
│ gp_flexible_baseline       │    711.59 │     2.22 │     28.6% │       33.0% │     ✓     │       20 │
│ gp_plus_fourier            │    736.29 │     4.22 │     50.8% │       62.3% │     ✗     │       24 │
└────────────────────────────┴───────────┴──────────┴───────────┴─────────────┴───────────┴──────────┘

Ground Truth Reference:

True marketing share: 100.0%

## Detailed ROAS Comparison

Let's examine per-channel ROAS accuracy for each strategy.

In [35]:
# Per-channel ROAS table
console = Console()
table = Table(
    title="Per-Channel ROAS Comparison", show_header=True, header_style="bold green"
)

table.add_column("Strategy", style="cyan")
table.add_column("Search-Ads", justify="right")
table.add_column("Error %", justify="right")
table.add_column("Social-Media", justify="right")
table.add_column("Error %", justify="right")
table.add_column("Local-Ads", justify="right")
table.add_column("Error %", justify="right")
table.add_column("Email", justify="right")
table.add_column("Error %", justify="right")

# Ground truth row
true_roas = ground_truth["roas_values"]["Local"]
table.add_row(
    "[bold]Ground Truth[/bold]",
    f"[bold]{true_roas['Search-Ads']:.2f}[/bold]",
    "-",
    f"[bold]{true_roas['Social-Media']:.2f}[/bold]",
    "-",
    f"[bold]{true_roas['Local-Ads']:.2f}[/bold]",
    "-",
    f"[bold]{true_roas['Email']:.2f}[/bold]",
    "-",
)

# Strategy rows
for row in results_df.iter_rows(named=True):
    if "error" not in row:
        table.add_row(
            row["strategy"],
            f"{row['roas_search_ads']:.2f}",
            f"{row['roas_search_ads_error_pct']:+.1f}%",
            f"{row['roas_social_media']:.2f}",
            f"{row['roas_social_media_error_pct']:+.1f}%",
            f"{row['roas_local_ads']:.2f}",
            f"{row['roas_local_ads_error_pct']:+.1f}%",
            f"{row['roas_email']:.2f}",
            f"{row['roas_email_error_pct']:+.1f}%",
        )

console.print("\n")
console.print(table)

                                            Per-Channel ROAS Comparison                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━━━┓
┃ Strategy                ┃ Search-Ads ┃ Error % ┃ Social-Media ┃ Error % ┃ Local-Ads ┃ Error % ┃ Email ┃ Error % ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━━━┩
│ Ground Truth            │       8.19 │       - │        21.95 │       - │      3.32 │       - │ 32.13 │       - │
│ static_intercept_only   │       4.43 │  +45.9% │        16.72 │  +23.8% │      2.84 │  +14.4% │ 31.13 │   +3.1% │
│ static_fourier_seasona… │       5.90 │  +28.0% │        23.93 │   +9.0% │      1.32 │  +60.4% │ 32.35 │   +0.7% │
│ gp_flexible_baseline    │       5.55 │  +32.2% │        24.88 │  +13.4% │      1.15 │  +65.5% │ 30.99 │   +3.5% │
│ gp_plus_fourier         │      17.69 │ +116.0% │        26.75 │  +21.9% │      1.20 │  +63.8% │ 32.59 │   +1.4% │
└─────────────────────────┴────────────┴─────────┴──────────────┴─────────┴───────────┴─────────┴───────┴─────────┘

## Save Results

In [36]:
# Save comparison results
output_dir = Path("../models/baseline_comparison")
output_dir.mkdir(parents=True, exist_ok=True)

# Save as CSV
results_df.write_csv(output_dir / "baseline_comparison_results.csv")

rprint(f"\n[bold green]Results saved to {output_dir}[/bold green]")

Results saved to ../models/baseline_comparison

## Summary

This notebook compared 4 baseline strategies for MMM:

1. **Static intercept only** - Constant baseline
2. **Static + Fourier seasonality** - Traditional approach
3. **GP flexible baseline** - Gaussian Process captures all variation
4. **GP + Fourier hybrid** - Combined approach

### Key Findings

Examine the results table above to identify:
- Which strategy has the **best test CRPS** (predictive performance)?
- Which strategy has the **best ROAS MAE** (causal accuracy)?
- Are they the same? If not, what's the trade-off?
- Does any strategy have suspiciously low marketing attribution share?

### Guidelines for Practitioners

Based on this analysis:

1. **Start with static + Fourier** for regular seasonal patterns
2. **Add GP** if you expect irregular events that can't be captured by Fourier
3. **Always validate** marketing attribution share (should be reasonable, ~30-50%)
4. **Check ROAS** against business intuition or prior studies
5. **Prefer interpretability** over small CRPS improvements if causal recovery suffers

### Next Steps

- Visualize baseline decompositions (see which strategies confound baseline with marketing)
- Fit final model with winner strategy on full dataset
- Extend to optimize saturation function parameters
- Implement budget allocation optimization